### **Importing the libraries**

In [ ]:
import os
import pandas as pd
import numpy as np
np.random.seed(1)
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import torch
torch.manual_seed(1)
from torch import nn
import torch.nn.functional as F
import torch.utils.data as data_utils
import torch.optim as optim
import random
random.seed(1)

# **1. Find the number of nodes in input and output layer according to the dataset and justify it in the report. Specify and justify any other hyper parameter that is/are needed.**



### **Function to read the dataset**

In [ ]:
def read_data():
    '''
        reads the dataset and returns a dataframe containing the required columns
    '''
    cols = ['lettr', 'x-box', 'y-box', 'width', 'high ', 'onpix', 'x-bar', 'y-bar', 'x2bar', 'y2bar', 'xybar', 'x2ybr', 'xy2br', 'x-ege', 'xegvy', 'y-ege', 'yegvx']

    df = pd.read_csv('letter-recognition.data', names=cols, header=None)

    label = dict()
    c = ord('A')
    i = 0
    
    while c <= ord('Z'):
        label[chr(c)] = i
        c += 1
        i += 1

    df['lettr'].replace(label, inplace=True)

    return df

data = read_data()

In [ ]:
X = data.iloc[:, 1:]
y = data['lettr']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=15)

scaling = StandardScaler() # MinMaxScaler()
scaling.fit(X_train)

X_train = scaling.transform(X_train)
X_train = pd.DataFrame(X_train)

X_test = scaling.transform(X_test)
X_test = pd.DataFrame(X_test)

In [ ]:
train_target = torch.tensor(y_train.values.astype(np.int))
train = torch.tensor(X_train.values.astype(np.float32))

train_tensor = data_utils.TensorDataset(train, train_target)
trainloader = data_utils.DataLoader(dataset=train_tensor, batch_size=128, shuffle=True)

test_target = torch.tensor(y_test.values.astype(np.int))
test = torch.tensor(X_test.values.astype(np.float32))

test_tensor = data_utils.TensorDataset(test, test_target)
testloader = data_utils.DataLoader(dataset=test_tensor, batch_size=1)

In [ ]:
learning_rates =  [0.1, 0.08, 0.05, 0.01, 0.001, 0.0001, 0.00001]
model_accuracies = dict()

## **Class for Neural Network Model**

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, layer_nodes):
        super(NeuralNetwork, self).__init__()
        self.n_layers = len(layer_nodes) - 1
        for i in range(len(layer_nodes)-1):
            layer_name = 'layer' + str(i)
            setattr(self, layer_name, nn.Linear(layer_nodes[i], layer_nodes[i+1]))
        
    def forward(self, x):
        i = 0
        while i < self.n_layers-1:
            layer_name = 'layer' + str(i)
            x = getattr(self, layer_name)(x)
            x = F.relu(x)
            i += 1
        layer_name = 'layer' + str(i)
        x = getattr(self, layer_name)(x)
        x = F.log_softmax(x, dim=1)
        return x

In [ ]:
class Model():
    def __init__(self, layers, learning_rate, epochs):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.model = NeuralNetwork(layers)
        # Define the loss
        self.criterion = nn.NLLLoss()
        # Optimizers require the parameters to optimize and a learning rate
        self.optimizer = optim.SGD(self.model.parameters(), lr=self.learning_rate)

    def fit(self, trainloader):
        for e in range(self.epochs):
            running_loss = 0
            for features, labels in trainloader:
                # features = features.view(-1, 16)
            
                # Training pass
                self.optimizer.zero_grad()
                
                output = self.model(features)
                loss = self.criterion(output, labels)
                loss.backward()
                self.optimizer.step()
                
                running_loss += loss.item()

            if (e+1)%100 == 0:
                print(f"Training loss after iteration {e+1}: {running_loss/len(trainloader)}")

    def accuracy(self, testloader):
        correct = 0
        total = 0

        with torch.no_grad():
            for X, y in testloader:
                output = self.model(X) # .view(-1, 16))
                for idx, i in enumerate(output):
                    if torch.argmax(i) == y[idx]:
                        correct += 1
                    total += 1
        
        return round(correct/total*100, 3)

# **2. Vary the number of hidden layers and number of nodes in each hidden layer. Consider the following architectures.**


## **Neural Network with 0 hidden layer**

In [ ]:
model_accuracies['Model 1'] = dict()
layer_nodes = [16, 26]

for learning_rate in learning_rates:
    model1 = Model(layers=layer_nodes, learning_rate=learning_rate, epochs=1000)
    model1.fit(trainloader)
    model1_accuracy = model1.accuracy(testloader)
    model_accuracies['Model 1'][learning_rate] = model1_accuracy
    print('Learning rate:', learning_rate, '\tTest Accuracy for Model 1:', model1_accuracy)

## **Neural Network with 1 hidden layer with 2 nodes**

In [ ]:
model_accuracies['Model 2'] = dict()
layer_nodes = [16, 2, 26]

for learning_rate in learning_rates:
    model2 = Model(layers=layer_nodes, learning_rate=learning_rate, epochs=1000)
    model2.fit(trainloader)
    model2_accuracy = model2.accuracy(testloader)
    model_accuracies['Model 2'][learning_rate] = model2_accuracy
    print('Learning rate:', learning_rate, '\tTest Accuracy for Model 2:', model2_accuracy)

## **Neural Network with 1 hidden layer with 6 nodes**

In [ ]:
model_accuracies['Model 3'] = dict()
layer_nodes = [16, 6, 26]

for learning_rate in learning_rates:
    model3 = Model(layers=layer_nodes, learning_rate=learning_rate, epochs=1000)
    model3.fit(trainloader)
    model3_accuracy = model3.accuracy(testloader)
    model_accuracies['Model 3'][learning_rate] = model3_accuracy
    print('Learning rate:', learning_rate, '\tTest Accuracy for Model 3:', model3_accuracy)

## **Neural Network with 2 hidden layers with 2 and 3 nodes respectively**

In [ ]:
model_accuracies['Model 4'] = dict()
layer_nodes = [16, 2, 3, 26]

for learning_rate in learning_rates:
    model4 = Model(layers=layer_nodes, learning_rate=learning_rate, epochs=1000)
    model4.fit(trainloader)
    model4_accuracy = model4.accuracy(testloader)
    model_accuracies['Model 4'][learning_rate] = model4_accuracy
    print('Learning rate:', learning_rate, '\tTest Accuracy for Model 4:', model4_accuracy)

## **Neural Network with 2 hidden layers with 3 and 2 nodes respectively**

In [ ]:
model_accuracies['Model 5'] = dict()
layer_nodes = [16, 3, 2, 26]

for learning_rate in learning_rates:
    model5 = Model(layers=layer_nodes, learning_rate=learning_rate, epochs=1000)
    model5.fit(trainloader)
    model5_accuracy = model5.accuracy(testloader)
    model_accuracies['Model 5'][learning_rate] = model5_accuracy
    print('Learning rate:', learning_rate, '\tTest Accuracy for Model 5:', model5_accuracy)

## **Neural Network with 2 hidden layers with 14 and 22 nodes respectively**

In [ ]:
model_accuracies['Model 6'] = dict()
layer_nodes = [16, 14, 22, 26] 

for learning_rate in learning_rates:
    model6 = Model(layers=layer_nodes, learning_rate=learning_rate, epochs=1000)
    model6.fit(trainloader)
    model6_accuracy = model6.accuracy(testloader)
    model_accuracies['Model 6'][learning_rate] = model6_accuracy
    print('Learning rate:', learning_rate, '\tTest Accuracy for Model:', model6_accuracy)

## **Printing the Model Accuracies for Different Learning Rates**

In [ ]:
models = []

for model in model_accuracies:
    print(model, ':', model_accuracies[model])
    models.append(model)

# **3. Plot graph for the results in the previous parts with respect to accuracy. (Learning rate vs accuracy for each model (in one plot) and model vs accuracy for each learning rate (in one plot).)**

In [ ]:
for model in models:
    x = [str(learning_rate) for learning_rate in learning_rates]
    y = [model_accuracies[model][learning_rate] for learning_rate in learning_rates]
    plt.plot(x, y, label=model)

plt.xlabel('Learning Rate')
plt.ylabel('Accuracy')
plt.title('Learning Rate VS Accuracy')
plt.legend()

plt.show()

In [ ]:
for learning_rate in learning_rates:
    x = [model for model in models]
    y = [model_accuracies[model][learning_rate] for model in models]
    plt.plot(x, y, label=learning_rate)

plt.xlabel('Model')
plt.ylabel('Accuracy')
plt.title('Model VS Accuracy')
plt.legend()

plt.show()

# **5. Reduce the feature dimension of the data into a two dimensional feature space using Principle Component Analysis (PCA). Plot the reduced dimensional data in a 2d plane. In the plot, all data points of a single class should have same color and data points from different classes should have different colors.**

In [ ]:
principal = PCA(n_components=2)
principal.fit(X_train)

X_train = principal.transform(X_train)
X_train = pd.DataFrame(X_train)

X_test = principal.transform(X_test)
X_test = pd.DataFrame(X_test)

In [ ]:
train_target = torch.tensor(y_train.values.astype(np.int))
train = torch.tensor(X_train.values.astype(np.float32))

train_tensor = data_utils.TensorDataset(train, train_target)
trainloader = data_utils.DataLoader(dataset=train_tensor, batch_size=128, shuffle=True)

test_target = torch.tensor(y_test.values.astype(np.int))
test = torch.tensor(X_test.values.astype(np.float32))

test_tensor = data_utils.TensorDataset(test, test_target)
testloader = data_utils.DataLoader(dataset=test_tensor, batch_size=1)

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(X_train.iloc[:, 0], X_train.iloc[:, 1], c=y_train, cmap='Set1')

plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('Principal Component Analysis')

plt.show()

# **6. Apply MLP of step 2 in the reduced feature space. Compare with classification output generated from step 2. you may use best learning rate obtained from step 3.**

In [ ]:
best_learning_rate = {'Model 1': 0.1, 'Model 2': 0.05, 'Model 3': 0.1, 'Model 4': 0.08, 'Model 5': 0.1, 'Model 6': 0.1}
model_accuracies_PCA = dict()

## **Neural Network with 0 hidden layer after applying PCA**

In [ ]:
model_accuracies_PCA['Model 1'] = dict()
layer_nodes = [2, 26]

learning_rate=best_learning_rate['Model 1']
model1 = Model(layers=layer_nodes, learning_rate=best_learning_rate['Model 1'], epochs=1000)
model1.fit(trainloader)
model1_accuracy = model1.accuracy(testloader)
model_accuracies_PCA['Model 1'][learning_rate] = model1_accuracy
print('Learning rate:', learning_rate, '\tTest Accuracy for Model 1:', model1_accuracy)

## **Neural Network with 1 hidden layer with 2 nodes after applying PCA**

In [ ]:
model_accuracies_PCA['Model 2'] = dict()
layer_nodes = [2, 2, 26]

learning_rate=best_learning_rate['Model 2']
model2 = Model(layers=layer_nodes, learning_rate=best_learning_rate['Model 2'], epochs=1000)
model2.fit(trainloader)
model2_accuracy = model2.accuracy(testloader)
model_accuracies_PCA['Model 2'][learning_rate] = model2_accuracy
print('Learning rate:', learning_rate, '\tTest Accuracy for Model 2:', model2_accuracy)

## **Neural Network with 1 hidden layer with 6 nodes after applying PCA**

In [ ]:
model_accuracies_PCA['Model 3'] = dict()
layer_nodes = [2, 6, 26]

learning_rate=best_learning_rate['Model 3']
model3 = Model(layers=layer_nodes, learning_rate=best_learning_rate['Model 3'], epochs=1000)
model3.fit(trainloader)
model3_accuracy = model3.accuracy(testloader)
model_accuracies_PCA['Model 3'][learning_rate] = model3_accuracy
print('Learning rate:', learning_rate, '\tTest Accuracy for Model 3:', model3_accuracy)

## **Neural Network with 2 hidden layers with 2 and 3 nodes respectively after applying PCA**

In [ ]:
model_accuracies_PCA['Model 4'] = dict()
layer_nodes = [2, 2, 3, 26]

learning_rate=best_learning_rate['Model 4']
model4 = Model(layers=layer_nodes, learning_rate=best_learning_rate['Model 4'], epochs=1000)
model4.fit(trainloader)
model4_accuracy = model4.accuracy(testloader)
model_accuracies_PCA['Model 4'][learning_rate] = model4_accuracy
print('Learning rate:', learning_rate, '\tTest Accuracy for Model 4:', model4_accuracy)

## **Neural Network with 2 hidden layers with 3 and 2 nodes respectively after applying PCA**

In [ ]:
model_accuracies_PCA['Model 5'] = dict()
layer_nodes = [2, 3, 2, 26]

learning_rate=best_learning_rate['Model 5']
model5 = Model(layers=layer_nodes, learning_rate=best_learning_rate['Model 5'], epochs=1000)
model5.fit(trainloader)
model5_accuracy = model5.accuracy(testloader)
model_accuracies_PCA['Model 5'][learning_rate] = model5_accuracy
print('Learning rate:', learning_rate, '\tTest Accuracy for Model 5:', model5_accuracy)

## **Neural Network with 2 hidden layers with 14 and 22 nodes respectively after applying PCA**

In [ ]:
model_accuracies_PCA['Model 6'] = dict()
layer_nodes = [2, 14, 22, 26]

learning_rate=best_learning_rate['Model 6']
model6 = Model(layers=layer_nodes, learning_rate=best_learning_rate['Model 6'], epochs=1000)
model6.fit(trainloader)
model6_accuracy = model6.accuracy(testloader)
model_accuracies_PCA['Model 6'][learning_rate] = model6_accuracy
print('Learning rate:', learning_rate, '\tTest Accuracy for Model 6:', model6_accuracy)